# 时间序列

时间序列数据是许多不同领域的结构化数据的重要形式，例如金融，经济学，生态学，神经科学和物理学。在许多时间点上观察或测量的任何东西都会形成一个时间序列。许多时间序列是固定频率的，也就是说，数据点根据某些规则以固定的间隔出现，例如每15秒，每5分钟或每月一次。时间序列也可以是不规则的，没有固定的时间单位或单位之间的偏移。如何标记和引用时间序列数据取决于应用程序，并且您可能具有以下之一：

- 时间戳记，特定时间点
- 固定时间段，例如2007年1月或2010年全年
- 时间间隔，以开始和结束时间戳记表示。句点可以被视为间隔的特殊情况
- 实验或经过的时间；每个时间戳是相对于特定开始时间的时间度量（例如，自放入烤箱以来每秒烘烤的曲奇的直径）

在本章中，我主要关注前三类中的时间序列，尽管许多技术都可以应用于实验时间序列，其中索引可以是整数或浮点数，指示从实验开始经过的时间。最简单和使用最广泛的时间序列是通过时间戳索引的时间序列。

> pandas还支持基于时间增量的索引，这可能是表示实验或经过时间的有用方式。 我们不会在本书中探讨timedelta索引，但是您可以在pandas文档中了解更多信息。

熊猫提供了许多内置的时间序列工具和数据算法。您可以有效地处理非常大的时间序列，并轻松地对不规则和固定频率的时间序列进行切片和切块，聚合以及重新采样。其中一些工具对于金融和经济应用程序特别有用，但是您当然也可以使用它们来分析服务器日志数据。

## 11.1 日期和时间数据类型和工具

Python标准库包括日期和时间数据的数据类型，以及与日历相关的功能。日期时间，时间和日历模块是开始的主要位置。datetime.datetime类型，或简称为datetime，被广泛使用：

In [1]:
from datetime import datetime

now = datetime.now()

now

datetime.datetime(2020, 7, 15, 18, 48, 27, 438797)

In [2]:
now.year, now.month, now.day

(2020, 7, 15)

datetime将日期和时间都存储到微秒。timedelta表示两个日期时间对象之间的时间差：

In [3]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)

delta

datetime.timedelta(days=926, seconds=56700)

In [4]:
delta.days

926

In [5]:
delta.seconds

56700

您可以向日期时间对象添加（或减去）一个时间增量或多个时间增量，以产生一个新的移位对象：

In [6]:
from datetime import timedelta

In [7]:
start = datetime(2011, 1, 7)

In [8]:
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [9]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

表11-1总结了datetime模块中的数据类型。尽管本章主要关注熊猫中的数据类型和更高级的时间序列操作，但您可能在Python的许多其他地方遇到了基于日期时间的类型。

### 在字符串和日期时间之间转换

您可以使用str或strftime方法格式化日期时间对象和pandas时间戳对象（稍后将介绍）为字符串，并传递格式说明：

In [10]:
stamp = datetime(2011, 1, 3)

In [12]:
str(stamp)

'2011-01-03 00:00:00'

In [13]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

有关格式代码的完整列表，请参见表11-2（从第2章中摘录）。

您可以使用以下相同的格式代码使用datetime.strptime将字符串转换为日期：

In [14]:
value = '2011-01-03'

In [15]:
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [16]:
datestrs = ['7/6/2011', '8/6/2011']

In [18]:
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

datetime.strptime是解析已知格式的日期的好方法。但是，每次都要编写格式规范可能会有些烦人，尤其是对于常见的日期格式。在这种情况下，您可以在第三方dateutil包中使用parser.parse方法（在安装熊猫时会自动安装该方法）：

In [19]:
from dateutil.parser import parse

In [20]:
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

dateutil能够解析大多数人类难以理解的日期表示形式：

In [21]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

在国际语言环境中，出现在月份之前的日期很常见，因此您可以通过dayfirst = True来表明这一点：

In [24]:
parse('6/12/2011', dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

pandas通常面向使用日期数组，无论是用作轴索引还是用作DataFrame中的列。to_datetime方法解析许多不同类型的日期表示形式。可以快速解析ISO 8601之类的标准日期格式：

In [25]:
datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']

In [27]:
import pandas as pd

pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

它还处理应视为缺失的值（无，空字符串等）：

In [28]:
idx = pd.to_datetime(datestrs + [None])

In [29]:
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [30]:
idx[2]

NaT

In [31]:
pd.isnull(idx)

array([False, False,  True])

NaT（不是时间）是时间戳数据的熊猫的null值。

> 警告：dateutil.parser是有用但不完善的工具。值得注意的是，它会将某些字符串识别为您可能不希望使用的日期，例如，“ 42”将被解析为具有今天日历日期的2042年。

datetime对象还具有许多其他国家或地区的语言的特定于区域设置的格式选项。例如，与英语系统相比，德语或法语系统上的缩写月份名称将有所不同。有关列表，请参见表11-3。

## 11.2 时间序列基础

pandas中一种基本的时间序列对象是一个以时间戳为索引的序列，该序列通常在pandas外部以Python字符串或日期时间对象表示：

In [32]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]

In [34]:
import numpy as np

ts = pd.Series(np.random.randn(6), index=dates)

In [35]:
ts

2011-01-02   -0.665978
2011-01-05    0.376143
2011-01-07   -0.560935
2011-01-08    0.542070
2011-01-10   -1.267143
2011-01-12    0.665247
dtype: float64

在后台，这些datetime对象已放入DatetimeIndex中：

In [36]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

与其他系列一样，索引不同的时间序列之间的算术运算会自动与日期对齐：

In [37]:
ts + ts[::2]

2011-01-02   -1.331955
2011-01-05         NaN
2011-01-07   -1.121870
2011-01-08         NaN
2011-01-10   -2.534285
2011-01-12         NaN
dtype: float64

回想一下ts[:: 2]选择ts中的第二个元素。

大熊猫使用NumPy的datetime64数据类型以十亿分之一秒的分辨率存储时间戳：

In [39]:
ts.index.dtype

dtype('<M8[ns]')

DatetimeIndex中的标量值是pandas Timestamp对象：

In [41]:
stamp = ts.index[0]

In [42]:
stamp

Timestamp('2011-01-02 00:00:00')

可以在使用datetime对象的任何地方替换时间戳。此外，它可以存储频率信息（如果有）并了解如何进行时区转换和其他类型的操作。稍后将进一步讨论这两个方面。

### 索引，选择，子集

当您基于标签索引和选择数据时，时间序列的行为与任何其他pandas.Series一样。

In [45]:
stamp = ts.index[2]

In [46]:
ts[stamp]

-0.5609348022030469

为方便起见，您还可以传递可解释为日期的字符串：

In [47]:
ts['1/10/2011']

-1.2671425880963991

In [48]:
ts['20110110']

-1.2671425880963991

对于更长的时间序列，可以传递一年或仅一年零一个月来轻松选择数据切片：

In [50]:
longer_ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))

In [51]:
longer_ts

2000-01-01    0.889725
2000-01-02   -0.000919
2000-01-03    0.509378
2000-01-04    1.913975
2000-01-05    1.371564
                ...   
2002-09-22   -0.171690
2002-09-23   -0.069877
2002-09-24   -0.876060
2002-09-25    1.260783
2002-09-26   -1.123596
Freq: D, Length: 1000, dtype: float64

In [52]:
longer_ts['2001']

2001-01-01    0.909263
2001-01-02   -1.480272
2001-01-03    0.217716
2001-01-04   -0.219772
2001-01-05    0.226118
                ...   
2001-12-27   -1.335583
2001-12-28   -0.948969
2001-12-29    0.020606
2001-12-30   -2.191826
2001-12-31    3.161953
Freq: D, Length: 365, dtype: float64

在这里，字符串“ 2001”被解释为年份，并选择该时间段。如果您指定月份，这也可以使用：

In [53]:
longer_ts['2001-05']

2001-05-01    0.787977
2001-05-02    0.118517
2001-05-03   -1.039016
2001-05-04   -1.201846
2001-05-05    0.196699
2001-05-06    1.311460
2001-05-07   -0.391598
2001-05-08    0.953764
2001-05-09    0.048582
2001-05-10   -1.539387
2001-05-11    0.771676
2001-05-12    1.223502
2001-05-13   -0.526161
2001-05-14   -0.408870
2001-05-15   -1.506422
2001-05-16    0.175692
2001-05-17   -0.383039
2001-05-18   -1.243778
2001-05-19   -0.413599
2001-05-20   -1.028607
2001-05-21   -2.091050
2001-05-22   -1.502383
2001-05-23    0.661210
2001-05-24    1.245096
2001-05-25   -0.979170
2001-05-26    0.097940
2001-05-27   -1.253084
2001-05-28   -0.123646
2001-05-29    0.922265
2001-05-30   -0.035729
2001-05-31   -1.256023
Freq: D, dtype: float64

用datetime对象切片也可以：

In [54]:
ts[datetime(2011, 1, 7)]

-0.5609348022030469

由于大多数时间序列数据都是按时间顺序排列的，因此可以使用时间序列中未包含的时间戳进行切片以执行范围查询：

In [55]:
ts

2011-01-02   -0.665978
2011-01-05    0.376143
2011-01-07   -0.560935
2011-01-08    0.542070
2011-01-10   -1.267143
2011-01-12    0.665247
dtype: float64

In [56]:
ts['1/6/2011':'1/11/2011']

2011-01-07   -0.560935
2011-01-08    0.542070
2011-01-10   -1.267143
dtype: float64

和以前一样，您可以传递字符串日期，日期时间或时间戳。请记住，以这种方式进行切片会像切片NumPy数组一样在源时间序列上产生视图。这意味着不会复制任何数据，并且片上的修改将反映在原始数据中。

有一个等效的实例方法truncate，它在两个日期之间切片一个Series：

In [57]:
ts.truncate(after='1/9/2011')

2011-01-02   -0.665978
2011-01-05    0.376143
2011-01-07   -0.560935
2011-01-08    0.542070
dtype: float64

所有这些对于DataFrame同样适用，在其行上建立索引：

In [59]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')

In [61]:
long_df = pd.DataFrame(np.random.randn(100, 4),
                       index=dates,
                       columns=['Colorado', 'Texas', 'New York', 'Ohio'])

In [62]:
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,1.493091,1.150157,-0.497640,-0.370084
2001-05-09,0.498095,0.067066,2.040200,2.132036
2001-05-16,-0.550955,-0.969391,0.153994,0.678015
2001-05-23,1.250802,2.204206,0.237780,0.093361
2001-05-30,-0.339419,-0.789699,-0.875361,-0.273705


### 具有重复索引的时间序列

在某些应用程序中，可能会有多个数据观测值落在特定的时间戳上。这是一个例子：

In [63]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])

In [64]:
dup_ts = pd.Series(np.arange(5), index=dates)

In [65]:
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

我们可以通过检查其is_unique属性来判断索引不是唯一的：

In [66]:
dup_ts.index.is_unique

False

现在，根据是否复制时间戳，索引到该时间序列将产生标量值或分片：

In [67]:
dup_ts['1/3/2000']

4

In [68]:
dup_ts['1/2/2000']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

假设您要聚合具有非唯一时间戳的数据。一种方法是使用groupby并通过level = 0：

In [69]:
grouped = dup_ts.groupby(level=0)

In [70]:
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

In [71]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64